# Progetto Finale Data Science - Travel

**Il tuo compito sarà seguire ogni punto e commentarlo, mostrandomi di aver capito cosa tu stia facendo. Se alcuni punti si rivelano impossibili spiegami la motivazione. Troverai probabilmente alcuni modelli o tecniche che non hai propriamente studiato. Non preoccuparti, la sperimentazione, la scoperta e il continuo aggiornamento fanno parte del gioco. CANCELLA QUESTO MESSAGGIO PRIMA DI CARICARE IL PROGETTO**

In questo progetto si analizzerà un dataset sulla soddisfazione dei clienti di una compagnia aerea.

l'obiettivo è **poter prevedere la soddisfazione degli utenti in base a dei parametri sulla soddisfazione dei servizi della compagnia aerea**

## EDA (Exploratory data analysis)

### DATA SELECTION

in questo progetto verrano caricati due dataset, uno per il training ed uno per il test, fornito dalla traccia su start2impact:

saranno caricati tutte le librerie python che verranno utilizzate in questo progetto:

In [2]:
import pandas as pd
from google.colab import files

pd.set_option('display.max_columns', None)

S'importa i due csv forniti dalla traccia su start2impact. E' stato utilizzato il codice sotto perchè per la creazione del progetto è stato utilizzando la piattaforma cloud di google Colab:

In [3]:
uploaded = files.upload()

Saving test.csv to test.csv
Saving train.csv to train.csv


In [4]:
df_test = pd.read_csv('test.csv')
df_train = pd.read_csv('train.csv')

Prima di iniziare ad analizzare il dataset bisogna capire quale dataset utilizzare o utilizzare entrambi. In questo progetto è stato deciso di utilizzare solo il training set in questa fase perchè con l'utilizzo del training set si evita di avere overfitting e data leakage (informazioni presente nel training set sono inseriti nel validation set o test set).

In [5]:
df_train.shape, df_test.shape

((103904, 25), (25976, 25))

si puoi analizzare che sommando la grandezza dei 2 dataset, il training set rappresenta il 75% dei dati mentre il training set rappresenta il 25% dei dati.

Viene dato un primo sguardo ai dati del dataset di training

In [6]:
df_train.head()

,Unnamed: 0,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,0,70172,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,3,1,5,3,5,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,1,5047,Male,disloyal Customer,25,Business travel,Business,235,3,2,3,3,1,3,1,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,2,110028,Female,Loyal Customer,26,Business travel,Business,1142,2,2,2,2,5,5,5,5,4,3,4,4,4,5,0,0.0,satisfied
3,3,24026,Female,Loyal Customer,25,Business travel,Business,562,2,5,5,5,2,2,2,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,4,119299,Male,Loyal Customer,61,Business travel,Business,214,3,3,3,3,4,5,5,3,3,4,4,3,3,3,0,0.0,satisfied


viene dato uno sguardo alle colonne presenti:

In [7]:
df_train.columns

Index(['Unnamed: 0', 'id', 'Gender', 'Customer Type', 'Age', 'Type of Travel',
       'Class', 'Flight Distance', 'Inflight wifi service',
       'Departure/Arrival time convenient', 'Ease of Online booking',
       'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort',
       'Inflight entertainment', 'On-board service', 'Leg room service',
       'Baggage handling', 'Checkin service', 'Inflight service',
       'Cleanliness', 'Departure Delay in Minutes', 'Arrival Delay in Minutes',
       'satisfaction'],
      dtype='object')

viene dato uno sguardo ai valori unici presenti in ogni colonna:

In [12]:
for colonna in df_train.columns:
    valori_unici = df_train[colonna].unique()
    print(f"Valori presenti nella '{colonna}' unici: {valori_unici}")

Valori presenti nella 'Unnamed: 0' unici: [     0      1      2 ... 103901 103902 103903]
Valori presenti nella 'id' unici: [ 70172   5047 110028 ...  68825  54173  62567]
Valori presenti nella 'Gender' unici: ['Male' 'Female']
Valori presenti nella 'Customer Type' unici: ['Loyal Customer' 'disloyal Customer']
Valori presenti nella 'Age' unici: [13 25 26 61 47 52 41 20 24 12 53 33 45 38  9 17 43 58 23 57 49 36 22 31
 15 35 67 37 40 34 39 50 29 54 21 28 27 69 60 48 59 46 30 66 64 44 51 32
 19 42 16 11 62  8 56 68 55 18 65 72 70 63 10  7 14 80 74 71 85 73 76 77
 75 79 78]
Valori presenti nella 'Type of Travel' unici: ['Personal Travel' 'Business travel']
Valori presenti nella 'Class' unici: ['Eco Plus' 'Business' 'Eco']
Valori presenti nella 'Flight Distance' unici: [ 460  235 1142 ...  974 1479  400]
Valori presenti nella 'Inflight wifi service' unici: [3 2 4 1 5 0]
Valori presenti nella 'Departure/Arrival time convenient' unici: [4 2 5 3 1 0]
Valori presenti nella 'Ease of Online booki

Dopo aver dato un primo sguardo al dataset, si può proseguire con il data cleaning:


Si procede a cancellare delle variabili che non sono indispensabili o rindondati come **Unnamed** perchè fornisce le stesse informazioni che ci da un dataframe di pandas e **id** perchè non è indispensabile per il progetto

In [18]:
df_train_feat_clean= df_train.drop(columns=["Unnamed: 0", "id"], axis=1)
df_train_feat_clean

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,3,1,5,3,5,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,Male,disloyal Customer,25,Business travel,Business,235,3,2,3,3,1,3,1,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,Female,Loyal Customer,26,Business travel,Business,1142,2,2,2,2,5,5,5,5,4,3,4,4,4,5,0,0.0,satisfied
3,Female,Loyal Customer,25,Business travel,Business,562,2,5,5,5,2,2,2,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,Male,Loyal Customer,61,Business travel,Business,214,3,3,3,3,4,5,5,3,3,4,4,3,3,3,0,0.0,satisfied
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103899,Female,disloyal Customer,23,Business travel,Eco,192,2,1,2,3,2,2,2,2,3,1,4,2,3,2,3,0.0,neutral or dissatisfied
103900,Male,Loyal Customer,49,Business travel,Business,2347,4,4,4,4,2,4,5,5,5,5,5,5,5,4,0,0.0,satisfied
103901,Male,disloyal Customer,30,Business travel,Business,1995,1,1,1,3,4,1,5,4,3,2,4,5,5,4,7,14.0,neutral or dissatisfied
103902,Female,disloyal Customer,22,Business travel,Eco,1000,1,1,1,5,1,1,1,1,4,5,1,5,4,1,0,0.0,neutral or dissatisfied


Si procede ad analizzare se ci sono dei valori nulli e come gestirli:

In [19]:
df_train_feat_clean.isna().sum()

Gender                                 0
Customer Type                          0
Age                                    0
Type of Travel                         0
Class                                  0
Flight Distance                        0
Inflight wifi service                  0
Departure/Arrival time convenient      0
Ease of Online booking                 0
Gate location                          0
Food and drink                         0
Online boarding                        0
Seat comfort                           0
Inflight entertainment                 0
On-board service                       0
Leg room service                       0
Baggage handling                       0
Checkin service                        0
Inflight service                       0
Cleanliness                            0
Departure Delay in Minutes             0
Arrival Delay in Minutes             310
satisfaction                           0
dtype: int64

In [20]:
df_train_feat_clean.isnull().sum()

Gender                                 0
Customer Type                          0
Age                                    0
Type of Travel                         0
Class                                  0
Flight Distance                        0
Inflight wifi service                  0
Departure/Arrival time convenient      0
Ease of Online booking                 0
Gate location                          0
Food and drink                         0
Online boarding                        0
Seat comfort                           0
Inflight entertainment                 0
On-board service                       0
Leg room service                       0
Baggage handling                       0
Checkin service                        0
Inflight service                       0
Cleanliness                            0
Departure Delay in Minutes             0
Arrival Delay in Minutes             310
satisfaction                           0
dtype: int64